In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os,sys
import PIL
import tensorflow as tf
import cv2 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from PIL import Image
from skimage.io import imread, imshow
from skimage.color import rgb2hsv
from matplotlib import pyplot as plt
from PIL import Image # importing image module 
import numpy as np 
import imageio
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image




In [116]:
full_dir = "E:/User Document/processed"
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  full_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)
class_names = train_ds.class_names
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 1120 files belonging to 6 classes.
Using 1008 files for training.
Found 1120 files belonging to 6 classes.
Using 112 files for validation.
['NitrogenHigh', 'NitrogenLow', 'PhosphorusHigh', 'PhosphorusLow', 'PotassiumHigh', 'PotassiumLow']


In [117]:


num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

print(model)

In [118]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 180, 180, 3)       0         
                                                                 
 conv2d_36 (Conv2D)          (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 90, 90, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 45, 45, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 45, 45, 64)      

In [119]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/10
32/32 [==============================] - 22s 603ms/step - loss: 1.6476 - accuracy: 0.3581 - val_loss: 1.5746 - val_accuracy: 0.3661
Epoch 2/10
32/32 [==============================] - 20s 626ms/step - loss: 1.3091 - accuracy: 0.4841 - val_loss: 1.4794 - val_accuracy: 0.4286
Epoch 3/10
32/32 [==============================] - 21s 667ms/step - loss: 1.1937 - accuracy: 0.5407 - val_loss: 1.3759 - val_accuracy: 0.4107
Epoch 4/10
32/32 [==============================] - 20s 611ms/step - loss: 1.1418 - accuracy: 0.5417 - val_loss: 1.2582 - val_accuracy: 0.4911
Epoch 5/10
32/32 [==============================] - 20s 623ms/step - loss: 1.0905 - accuracy: 0.5565 - val_loss: 1.1010 - val_accuracy: 0.6250
Epoch 6/10
32/32 [==============================] - 20s 608ms/step - loss: 1.0330 - accuracy: 0.5942 - val_loss: 1.2128 - val_accuracy: 0.5089
Epoch 7/10
32/32 [==============================] - 18s 569ms/step - loss: 0.9677 - accuracy: 0.6200 - val_loss: 0.9619 - val_accuracy: 0.6250

In [120]:
from PIL import Image
def load_image(img_path, show=False):

    img = Image.open(img_path)
    #img = img1.Rescaling(1./255, input_shape=(img_height, img_width, 3))
    img_tensor = np.array(img)                    #       # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 1./255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [144]:
from PIL import Image
image = Image.open('E:/User Document/processed/PotassiumHigh/PotassiumHigh44.jpg')
image = image.resize((180, 180))
image = np.array(image)
img_tensor = np.expand_dims(image, axis=0)    
model.predict(img_tensor)

array([[-3.1902034,  0.6879859, -1.9124906,  0.8053539,  2.5274453,
         1.9255642]], dtype=float32)

In [150]:
model.save('E:/User Document/my_model.h5') 